In [ ]:
from google.colab import drive
drive.mount('./content')

In [1]:
import keras
keras.__version__

'2.4.3'

- 생성자

In [2]:
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# 입력을 16 × 16 크기의 128개 채널을 가진 특성 맵으로 변환합니다
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# 합성곱 층을 추가합니다
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기로 업샘플링합니다
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# 합성곱 층을 더 추가합니다
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기의 1개 채널을 가진 특성 맵을 생성합니다
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)      

- 판별자
    - : 후보이미지(진짜 또는 가짜)를 입력으로 받고 2개의 클래스로 분류
    - 클래스는 '생성된 이미지' 또는 '훈련세트에서 온 진짜 이미지' 이다.


In [25]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.3)(x)       # droput 4개 추가
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)

# 드롭아웃 층을 넣는 것이 아주 중요합니다!
x = layers.Dropout(0.4)(x)

# 분류 층
x = layers.Dense(1, activation='sigmoid')(x)

# (32,32, 3) 크기의 입력을 이진분류 결정(진짜/가짜)로 변환하는 판별자 모델 객체 생성
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()


discriminator_optimizer = keras.optimizers.RMSprop(lr=0.00008  # 학습률 낮춤
                                                   , clipvalue=1.0 # 옵티마이저에서 (값을 지정하여) 그래디언트 클리핑을 사용 
                                                   , decay=1e-8)   # 안정된 훈련을 위해서 학습률 감쇠를 사용합니다
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

TypeError: __init__() missing 1 required positional argument: 'rate'

- 적대적 네트워크
    - 생성자와 판별자를 연결하여 GAN을 설정
    - 훈련 시, 생성자가 판별자를 속이는 방향으로 학습
    - 이 모델은 잠재 공간의 포인트를 "진짜" 또는 "가짜"의 분류 결정으로 변환
    - 훈련에 사용되는 타깃 레이블은 항상 "진짜 이미지"이다.
    - 훈련하는 동안 판별자를 동결하는 것이 중요. (학습되지 않도록. 가중치가 업데이트되지 않도록)


- : GAN을 훈련하는 것은 DISCRIMINATOR가 가짜 이미지를 보고 진짜라고 예측하도록 만들기 위해 GENERATOR의 가중치를 업데이트.



In [4]:
# 판별자의 가중치가 훈련되지 않도록 설정합니다(gan 모델에만 적용됩니다)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

- 데이터 로드

In [5]:
import os
from keras.preprocessing import image

In [13]:
img_dir = "../input/convert-image"

# img_dir = '/content/content/MyDrive/Colab Notebooks/sai/10img.zip (Unzipped Files)'

In [8]:
from PIL import Image
from numpy import asarray
x_train = []

In [14]:
for i in os.listdir(img_dir):
  # load the image
  image = Image.open(os.path.join(img_dir, i))
  # convert image to numpy array
  data = asarray(image)
  x_train.append(data)

x_train= asarray(x_train)
x_train = x_train/255.    #정규화

In [15]:
x_train= asarray(x_train)

In [16]:
x_train.shape # (504개의 이미지, 32, 32, 3채널)

(504, 32, 32, 3)

In [17]:
# 데이터를 정규화합니다
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

- GAN 훈련 구현


- 훈련시, 적대적 손실이 크게 증가한다면(= 판별자의 손실은 0으로 향하는 경우), 판별자의 학습률을 낮추고 판별자의 드롭아웃 비율을 높이기

In [18]:
iterations = 100
batch_size = 64

# 생성된 이미지 저장 위치 지정
save_dir = './gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images]) 

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    # generate 1, real 0
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다 (사실은 거짓)
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다 (100번 스텝마다)
    if step % 10 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))

스텝 0에서 판별자 손실: 0.6932320594787598
스텝 0에서 적대적 손실: 0.7532095313072205
스텝 10에서 판별자 손실: 0.5282623171806335
스텝 10에서 적대적 손실: 2.428219795227051
스텝 20에서 판별자 손실: 0.6990195512771606
스텝 20에서 적대적 손실: 0.7936890125274658
스텝 30에서 판별자 손실: 0.697198212146759
스텝 30에서 적대적 손실: 0.7597161531448364
스텝 40에서 판별자 손실: 0.6946892738342285
스텝 40에서 적대적 손실: 0.7197667360305786
스텝 50에서 판별자 손실: 0.4044938087463379
스텝 50에서 적대적 손실: 4.474719047546387
스텝 60에서 판별자 손실: 0.23664575815200806
스텝 60에서 적대적 손실: 7.416868209838867
스텝 70에서 판별자 손실: 0.11404924094676971
스텝 70에서 적대적 손실: 28.40412712097168
스텝 80에서 판별자 손실: 0.25061625242233276
스텝 80에서 적대적 손실: 2.069443702697754
스텝 90에서 판별자 손실: 0.5029643177986145
스텝 90에서 적대적 손실: 11.750680923461914


In [ ]:
'''
import time

iterations = 1000
batch_size = 64
save_dir = './gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    s = time.time()
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다
    if step % 100 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))
    e = time.time()
    print('%d번째 소요시간: ' %step ,end='')
    print(e-s)

'''

In [20]:
import matplotlib.pyplot as plt

In [21]:
# 잠재 공간에서 랜덤한 포인트를 샘플링합니다
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# 가짜 이미지로 디코딩합니다
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()

AttributeError: 'JpegImageFile' object has no attribute 'array_to_img'